**bold text**

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

WE CAN USE THIS METHOD TO HELP US SELECT THE DATA DIRECTLY FROM THE LOCAL MACHINE INTO GOOGLE COLAB ENVIROMENT TO HELP US IN THE ANALYSIS AND OTHER MACHINE LEARNING MODELS.

In [ ]:
try:
    from google.colab import files
    uploaded_files = files.upload()
    zip_file_path = next(iter(uploaded_files))
    print("File uploaded successfully:", zip_file_path)
except ImportError as e:
    print("Google Colab is not available. Please run this code in a Google Colab environment.")
except Exception as e:
    print("An error occurred:", e)

Saving 10253.zip to 10253 (2).zip
File uploaded successfully: 10253 (2).zip


In [ ]:
# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get a list of all files in the zip
    files = zip_ref.namelist()

    # Loop through the files
    for file in files:
        # Check if the file is a directory (subfolder like 10253, 10254, etc.)
        if file.endswith('/'):
            subdir = file.split('/')[-2]
            # Loop through the subfolders (0 and 1)
            for label in ['0/', '1/']:
                label_path = os.path.join(subdir, label)
                if label_path in files:
                    # Loop through the image files
                    for img_file in zip_ref.namelist():
                        if img_file.startswith(label_path) and not img_file.endswith('/'):
                            image_paths.append(img_file)
                            labels.append(int(label.strip('/')))

In [ ]:
# Create a DataFrame
data = pd.DataFrame({"image_path": image_paths, "label": labels})

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Check if the data array is empty
if len(data["image_path"].values) == 0:
    print("Error: No data available for splitting.")
else:
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(data["image_path"].values, data["label"].values, test_size=0.2, random_state=42)

Error: No data available for splitting.


In [ ]:
# Data preprocessing
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_dataframe(
    dataframe=pd.DataFrame({"image_path": X_train, "label": y_train}),
    directory=None,
    x_col="image_path",
    y_col="label",
    batch_size=32,
    class_mode="binary",
    target_size=(50, 50))

test_generator = data_generator.flow_from_dataframe(
    dataframe=pd.DataFrame({"image_path": X_test, "label": y_test}),
    directory=None,
    x_col="image_path",
    y_col="label",
    batch_size=32,
    class_mode="binary",
    target_size=(50, 50))

NameError: name 'X_train' is not defined

In [ ]:
# CapsNet model architecture
def CapsNet(input_shape, n_class, routings):
    x = Input(shape=input_shape)

    # Layer 1: Conv2D layer with `conv` kernel
    conv1 = Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Primary Capsule layer
    primaryCaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer
    digitCaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='digitcaps')(primaryCaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length
    out_caps = Length(name='capsnet')(digitCaps)

    # Decoder network
    y = Input(shape=(n_class,))
    masked_by_y = Mask()([digitCaps, y])
    decoder = Sequential([Dense(512, activation='relu', input_dim=16 * n_class),
                          Dense(1024, activation='relu'),
                          Dense(np.prod(input_shape), activation='sigmoid')])

    # Model
    model = Model([x, y], [out_caps, decoder(masked_by_y)])

    return model

# CapsNet model
model = CapsNet(input_shape=(50, 50, 3), n_class=2, routings=3)
model.compile(optimizer=Adam(lr=0.001),
              loss=['binary_crossentropy', 'mse'],
              loss_weights=[1., 0.392],
              metrics={'capsnet': 'accuracy'})

# Train the CapsNet model
epochs = 50
batch_size = 32

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training
history = model.fit(train_generator, epochs=epochs, validation_data=test_generator, callbacks=[early_stopping])

# Evaluate the CapsNet model
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred[0], axis=-1)
y_true = test_generator.classes

print("CapsNet Classification Report:")
print(classification_report(y_true, y_pred))

# Visualizations
# 1. Training and validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['capsnet_accuracy'])
plt.plot(history.history['val_capsnet_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# 2. Training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# 3. Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.xticks([], [])
plt.yticks([], [])
plt.title('Confusion Matrix')
plt.show()

# 4. ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# 5. Visualize some misclassified images
misclassified_idx = np.where(y_pred != y_true)[0]
misclassified_images = X_test.iloc[misclassified_idx]

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
ax = ax.flatten()

for i, idx in enumerate(misclassified_idx[:6]):
    img = plt.imread(f"../input/breast-histopathology-images/images/{misclassified_images.iloc[i]['image_path']}")
    ax[i].imshow(img)
    ax[i].set_title(f"Predicted: {y_pred[idx]}, Actual: {y_true[idx]}")
    ax[i].axis('off')

plt.tight_layout()
plt.show()

# Compare with traditional models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Convert data to numpy arrays
X_train_np = np.array([plt.imread(f"../input/breast-histopathology-images/images/{path}") for path in X_train["image_path"]])
X_test_np = np.array([plt.imread(f"../input/breast-histopathology-images/images/{path}") for path in X_test["image_path"]])

# Flatten the images
X_train_np = X_train_np.reshape(X_train_np.shape[0], -1)

In [ ]:
# Traditional models
models = [
   LogisticRegression(),
   KNeighborsClassifier(),
   DecisionTreeClassifier(),
   RandomForestClassifier(),
   SVC()
]

model_names = [
   'Logistic Regression',
   'K-Nearest Neighbors',
   'Decision Tree',
   'Random Forest',
   'Support Vector Machine'
]

for model, name in zip(models, model_names):
   model.fit(X_train_np, y_train)
   y_pred = model.predict(X_test_np)
   print(f"{name} Classification Report:")
   print(classification_report(y_true, y_pred))
   print()

# Additional visualizations for traditional models

# 6. Accuracy comparison
accuracies = []
for model in models:
   model.fit(X_train_np, y_train)
   y_pred = model.predict(X_test_np)
   accuracy = (y_pred == y_true).mean()
   accuracies.append(accuracy)

plt.figure(figsize=(8, 6))
plt.bar(model_names, accuracies)
plt.title('Model Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.show()

# 7. ROC curve comparison
plt.figure(figsize=(8, 6))
for model, name in zip(models, model_names):
   model.fit(X_train_np, y_train)
   y_score = model.predict_proba(X_test_np)[:, 1]
   fpr, tpr, _ = roc_curve(y_true, y_score)
   roc_auc = auc(fpr, tpr)
   plt.plot(fpr, tpr, lw=2, label=f'{name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()